# 📦 YAML Includes & Pipeline Composition

Learn how to build modular, reusable YAML pipelines using the `!include` directive:

- **Modular workflows** - Break large pipelines into reusable components
- **DRY principle** - Don't Repeat Yourself with shared configurations
- **Team collaboration** - Share common nodes across projects
- **Environment configs** - Different settings per environment (dev/staging/prod)
- **Security** - Built-in protection against directory traversal and circular includes

The `!include` directive enables composition patterns similar to Kubernetes ConfigMaps and Helm values files.

## Why Use YAML Includes?

**Without Includes:**
```yaml
# pipeline-a.yaml - 200 lines
nodes:
  - common_node_1: { ... 20 lines ... }
  - common_node_2: { ... 20 lines ... }
  - specific_node_a: { ... }

# pipeline-b.yaml - 200 lines  
nodes:
  - common_node_1: { ... 20 lines DUPLICATED ... }
  - common_node_2: { ... 20 lines DUPLICATED ... }
  - specific_node_b: { ... }
```

**With Includes:**
```yaml
# common-nodes.yaml - 40 lines
common_nodes: &common
  - common_node_1: { ... 20 lines ... }
  - common_node_2: { ... 20 lines ... }

# pipeline-a.yaml - 10 lines
nodes:
  - !include common-nodes.yaml#common
  - specific_node_a: { ... }

# pipeline-b.yaml - 10 lines
nodes:
  - !include common-nodes.yaml#common
  - specific_node_b: { ... }
```

**Benefits:**
- ✅ **90% less duplication** - Single source of truth
- ✅ **Easier maintenance** - Update once, affects all pipelines
- ✅ **Better organization** - Logical separation of concerns
- ✅ **Team scalability** - Teams can share component libraries

## Setup: Create Temporary Directory Structure

We'll create a realistic project structure with shared components:

In [ ]:
import tempfile
from pathlib import Path

# Create temporary directory for our examples
temp_dir = Path(tempfile.mkdtemp())
print(f"📁 Working directory: {temp_dir}")

# Create project structure
(temp_dir / "shared").mkdir()
(temp_dir / "pipelines").mkdir()
(temp_dir / "environments").mkdir()

# Create helper functions file for examples
helper_functions = '''
"""Helper functions for YAML include examples."""

def validate_string(data: str) -> str:
    """Validate that input is a string."""
    return str(data)

def clean_whitespace(data: str) -> str:
    """Remove leading/trailing whitespace."""
    return data.strip()

def to_uppercase(data: str) -> str:
    """Convert string to uppercase."""
    return data.upper()

def to_lowercase(data: str) -> str:
    """Convert string to lowercase."""
    return data.lower()

def calculate_length(data: str) -> int:
    """Calculate string length."""
    return len(data)
'''

(temp_dir / "helpers.py").write_text(helper_functions)

# Add temp_dir to Python path so we can import helpers
import sys

sys.path.insert(0, str(temp_dir))

print("\n✅ Directory structure created:")
print("   shared/       - Reusable components")
print("   pipelines/    - Main pipeline definitions")
print("   environments/ - Environment-specific configs")
print("   helpers.py    - Helper functions for examples")

## Example 1: Simple Include - Shared Nodes

The most common pattern: extracting reusable nodes into a shared file.

In [ ]:
# Create shared nodes file
shared_nodes_yaml = """
# Common validation and preprocessing nodes
validation_nodes:
  - kind: function_node
    metadata:
      name: input_validator
    spec:
      fn: "helpers.validate_string"
      dependencies: []

  - kind: function_node
    metadata:
      name: data_cleaner
    spec:
      fn: "helpers.clean_whitespace"
      dependencies: [input_validator]
"""

(temp_dir / "shared" / "common-nodes.yaml").write_text(shared_nodes_yaml)
print("✅ Created: shared/common-nodes.yaml")
print("\n📄 Contents:")
print(shared_nodes_yaml)

In [ ]:
# Create pipeline that includes shared nodes
# When including a list into a list, the !include should be a list item
pipeline_with_include = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: text-processor
  description: Process text using shared validation nodes
spec:
  # The !include will expand to a list of nodes
  nodes: {"!include": "shared/common-nodes.yaml#validation_nodes"}
"""

(temp_dir / "pipelines" / "text-processor.yaml").write_text(pipeline_with_include)

# Also copy shared nodes to pipelines/shared for this example
(temp_dir / "pipelines" / "shared").mkdir()
import shutil

shutil.copy(
    temp_dir / "shared" / "common-nodes.yaml",
    temp_dir / "pipelines" / "shared" / "common-nodes.yaml",
)

print("✅ Created: pipelines/text-processor.yaml")
print("✅ Copied: shared/common-nodes.yaml → pipelines/shared/common-nodes.yaml")
print("\n📄 Pipeline Contents:")
print(pipeline_with_include)
print("\n💡 The !include directive will expand to the full list of validation nodes")

### Execute Pipeline With Includes

The builder automatically resolves `!include` directives:

In [ ]:
from hexdag.core.orchestration.orchestrator import Orchestrator
from hexdag.core.pipeline_builder.yaml_builder import YamlPipelineBuilder

# Build pipeline (includes are automatically resolved)
# Note: build_from_yaml_file() automatically sets base_path to the YAML file's directory
builder = YamlPipelineBuilder(base_path=temp_dir)
graph, config = builder.build_from_yaml_file(str(temp_dir / "pipelines" / "text-processor.yaml"))

print("🏗️  Pipeline built successfully!")
print(f"   Nodes: {list(graph.nodes.keys())}")
print(f"   Total nodes: {len(graph.nodes)}")
print("\n💡 Notice: Included nodes (input_validator, data_cleaner) are merged seamlessly!")

In [ ]:
# Execute the pipeline
orchestrator = Orchestrator()
result = await orchestrator.run(graph, "  hello world  ")

print("📊 Execution Results:")
print("   Input: '  hello world  '")
print(f"   After validation: '{result.get('input_validator')}'")
print(f"   After cleaning: '{result.get('data_cleaner')}'")
print(f"   Final output: '{result.get('text_processor')}'")
print("\n✅ Include directive successfully resolved and executed!")

## Example 2: Nested Includes - Multi-Level Composition

Includes can reference other files with includes, enabling hierarchical composition:

In [ ]:
# Level 1: Base validation (create in pipelines/shared/)
base_validation = """
base_validators:
  - kind: function_node
    metadata:
      name: type_checker
    spec:
      fn: "helpers.to_lowercase"
      dependencies: []
"""

(temp_dir / "pipelines" / "shared" / "base-validation.yaml").write_text(base_validation)
print("✅ Created: pipelines/shared/base-validation.yaml (Level 1)")

In [ ]:
# Level 2: Extended validation (includes base, stays in shared/)
extended_validation = """
extended_validators:
  - {"!include": "base-validation.yaml#base_validators"}
  
  # Add extended validation
  - kind: function_node
    metadata:
      name: length_checker
    spec:
      fn: "helpers.calculate_length"
      dependencies: [type_checker]
"""

(temp_dir / "pipelines" / "shared" / "extended-validation.yaml").write_text(extended_validation)
print("✅ Created: pipelines/shared/extended-validation.yaml (Level 2 - includes Level 1)")

In [ ]:
# Level 3: Final pipeline (includes extended, relative to pipelines/)
nested_pipeline = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: nested-validation
  description: Pipeline with nested includes
spec:
  nodes:
    - {"!include": "shared/extended-validation.yaml#extended_validators"}
    
    # Add final processing - depends on type_checker from level 1
    - kind: function_node
      metadata:
        name: result_formatter
      spec:
        fn: "helpers.to_uppercase"
        dependencies: [type_checker]
"""

(temp_dir / "pipelines" / "nested-validation.yaml").write_text(nested_pipeline)
print("✅ Created: pipelines/nested-validation.yaml (Level 3 - includes Level 2)")
print("\n🔗 Include Chain:")
print("   pipelines/nested-validation.yaml")
print("   └── includes: shared/extended-validation.yaml")
print("       └── includes: shared/base-validation.yaml")
print("\n💡 All files are within pipelines/ directory - no security violations!")

In [ ]:
# Build and verify nested includes work
builder = YamlPipelineBuilder(base_path=temp_dir)
graph, config = builder.build_from_yaml_file(str(temp_dir / "pipelines" / "nested-validation.yaml"))

print("🏗️  Nested includes resolved successfully!")
print(f"   Total nodes: {len(graph.nodes)}")
print(f"   Node chain: {list(graph.nodes.keys())}")
print("\n✅ All 3 levels of includes merged correctly!")

## Example 3: Environment-Specific Configs

Use includes to maintain environment-specific settings (dev/staging/prod):

In [ ]:
# Shared pipeline structure
base_pipeline = """
base_nodes:
  - kind: function_node
    metadata:
      name: processor
    spec:
      fn: "helpers.to_uppercase"
      dependencies: []
"""

(temp_dir / "shared" / "base-pipeline.yaml").write_text(base_pipeline)
print("✅ Created: shared/base-pipeline.yaml")

In [ ]:
# Development environment config
dev_config = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: app-pipeline
  namespace: development
  description: Development environment - verbose logging
spec:
  nodes:
    - {"!include": "../shared/base-pipeline.yaml#base_nodes"}
  
  # Dev-specific: Add debug logging
  policies:
    logging:
      level: DEBUG
      verbose: true
"""

(temp_dir / "environments" / "dev.yaml").write_text(dev_config)
print("✅ Created: environments/dev.yaml")

In [ ]:
# Production environment config
prod_config = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: app-pipeline
  namespace: production
  description: Production environment - minimal logging
spec:
  nodes:
    - {"!include": "../shared/base-pipeline.yaml#base_nodes"}
  
  # Prod-specific: Minimal logging, monitoring enabled
  policies:
    logging:
      level: ERROR
      verbose: false
    monitoring:
      enabled: true
      alerts: true
"""

(temp_dir / "environments" / "prod.yaml").write_text(prod_config)
print("✅ Created: environments/prod.yaml")
print("\n💡 Same base pipeline, different environment configs!")

In [ ]:
# Load development config
builder = YamlPipelineBuilder(base_path=temp_dir)
graph_dev, config_dev = builder.build_from_yaml_file(str(temp_dir / "environments" / "dev.yaml"))

print("🔧 Development Environment:")
print(f"   Namespace: {config_dev.metadata.get('namespace')}")
print(f"   Logging level: {config_dev.policies.get('logging', {}).get('level')}")
print(f"   Verbose: {config_dev.policies.get('logging', {}).get('verbose')}")
print(f"   Nodes: {list(graph_dev.nodes.keys())}")

In [ ]:
# Load production config
graph_prod, config_prod = builder.build_from_yaml_file(str(temp_dir / "environments" / "prod.yaml"))

print("🚀 Production Environment:")
print(f"   Namespace: {config_prod.metadata.get('namespace')}")
print(f"   Logging level: {config_prod.policies.get('logging', {}).get('level')}")
print(f"   Monitoring: {config_prod.policies.get('monitoring', {}).get('enabled')}")
print(f"   Nodes: {list(graph_prod.nodes.keys())}")
print("\n✅ Same pipeline structure, different configurations!")

## Example 4: Security Features

The include system has built-in security protections:

### 4a. Circular Include Detection

Prevents infinite loops from circular references:

In [ ]:
# Create circular reference
circular_a = """
nodes_a: &a
  - {"!include": "circular-b.yaml#nodes_b"}
"""

circular_b = """
nodes_b: &b
  - {"!include": "circular-a.yaml#nodes_a"}
"""

(temp_dir / "shared" / "circular-a.yaml").write_text(circular_a)
(temp_dir / "shared" / "circular-b.yaml").write_text(circular_b)

print("✅ Created circular reference files")
print("   circular-a.yaml → circular-b.yaml")
print("   circular-b.yaml → circular-a.yaml")

In [ ]:
# Try to build pipeline with circular include
circular_pipeline = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: circular-test
spec:
  nodes:
    - {"!include": "../shared/circular-a.yaml#nodes_a"}
"""

(temp_dir / "pipelines" / "circular-test.yaml").write_text(circular_pipeline)

try:
    builder = YamlPipelineBuilder(base_path=temp_dir)
    graph, config = builder.build_from_yaml_file(str(temp_dir / "pipelines" / "circular-test.yaml"))
    print("❌ Should have detected circular include!")
except Exception as e:
    print("✅ Circular include detected and prevented!")
    print(f"   Error: {type(e).__name__}")
    print(f"   Message: {str(e)[:100]}...")

### 4b. Directory Traversal Protection

Prevents malicious paths from accessing files outside the project:

In [ ]:
# Try to access file outside project directory
malicious_pipeline = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: malicious-test
spec:
  nodes:
    # Try to traverse outside project
    - {"!include": "../../../../etc/passwd"}
"""

(temp_dir / "pipelines" / "malicious-test.yaml").write_text(malicious_pipeline)

try:
    builder = YamlPipelineBuilder(base_path=temp_dir)
    graph, config = builder.build_from_yaml_file(
        str(temp_dir / "pipelines" / "malicious-test.yaml")
    )
    print("❌ Should have blocked directory traversal!")
except Exception as e:
    print("✅ Directory traversal blocked!")
    print(f"   Error: {type(e).__name__}")
    print(f"   Message: {str(e)[:100]}...")

### 4c. Absolute Path Rejection

Only relative paths are allowed for security:

In [ ]:
# Try to use absolute path
absolute_pipeline = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: absolute-test
spec:
  nodes:
    - {"!include": "/absolute/path/to/file.yaml"}
"""

(temp_dir / "pipelines" / "absolute-test.yaml").write_text(absolute_pipeline)

try:
    builder = YamlPipelineBuilder(base_path=temp_dir)
    graph, config = builder.build_from_yaml_file(str(temp_dir / "pipelines" / "absolute-test.yaml"))
    print("❌ Should have rejected absolute path!")
except Exception as e:
    print("✅ Absolute path rejected!")
    print(f"   Error: {type(e).__name__}")
    print(f"   Message: {str(e)[:100]}...")

## Example 5: Real-World Pattern - Component Library

Build a reusable component library for your organization:

In [ ]:
# Create component library structure within pipelines directory
(temp_dir / "pipelines" / "components").mkdir()
(temp_dir / "pipelines" / "components" / "validation").mkdir()
(temp_dir / "pipelines" / "components" / "processing").mkdir()
(temp_dir / "pipelines" / "components" / "output").mkdir()

print("📚 Component Library Structure:")
print("   pipelines/")
print("   └── components/")
print("       ├── validation/  - Input validation components")
print("       ├── processing/  - Data processing components")
print("       └── output/      - Output formatting components")

In [ ]:
# Validation component
validation_component = """
string_validators:
  - kind: function_node
    metadata:
      name: string_validator
    spec:
      fn: "helpers.validate_string"
      dependencies: []
  
  - kind: function_node
    metadata:
      name: trim_whitespace
    spec:
      fn: "helpers.clean_whitespace"
      dependencies: [string_validator]
"""

(temp_dir / "pipelines" / "components" / "validation" / "string-validation.yaml").write_text(
    validation_component
)
print("✅ Created: pipelines/components/validation/string-validation.yaml")

In [ ]:
# Processing component
processing_component = """
text_processors: &processors
  - kind: function_node
    metadata:
      name: to_uppercase
    spec:
      fn: "str.upper"
    dependencies: []
  
  - kind: function_node
    metadata:
      name: get_length
    spec:
      fn: "builtins.len"
    dependencies: []
"""

(temp_dir / "pipelines" / "components" / "processing" / "text-processing.yaml").write_text(
    processing_component
)
print("✅ Created: pipelines/components/processing/text-processing.yaml")

In [ ]:
# Output component
output_component = """
formatters:
  - kind: function_node
    metadata:
      name: format_output
    spec:
      fn: "helpers.to_uppercase"
      dependencies: []
"""

(temp_dir / "pipelines" / "components" / "output" / "formatters.yaml").write_text(output_component)
print("✅ Created: pipelines/components/output/formatters.yaml")

In [ ]:
# Compose pipeline from component library
composed_pipeline = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: composed-from-library
  description: Pipeline composed from reusable components
spec:
  nodes:
    # Import validation components
    - {"!include": "components/validation/string-validation.yaml#string_validators"}
    
    # Import processing components  
    - kind: function_node
      metadata:
        name: process_text
      spec:
        fn: "helpers.to_uppercase"
        dependencies: [trim_whitespace]
    
    # Import output components
    - kind: function_node
      metadata:
        name: final_output
      spec:
        fn: "helpers.validate_string"
        dependencies: [process_text]
"""

(temp_dir / "pipelines" / "composed-pipeline.yaml").write_text(composed_pipeline)
print("✅ Created: pipelines/composed-pipeline.yaml")
print("\n🔧 Pipeline composed from component library!")
print("   All includes are relative to pipelines/ directory")

In [ ]:
# Build and execute composed pipeline
builder = YamlPipelineBuilder(base_path=temp_dir)
graph, config = builder.build_from_yaml_file(str(temp_dir / "pipelines" / "composed-pipeline.yaml"))

print("🏗️  Composed Pipeline Built:")
print(f"   Total nodes: {len(graph.nodes)}")
print(f"   Nodes: {list(graph.nodes.keys())}")
print("\n📊 Component Sources:")
print("   • string_validator, trim_whitespace → validation/string-validation.yaml")
print("   • process_text → defined inline")
print("   • final_output → defined inline")

# Execute
orchestrator = Orchestrator()
result = await orchestrator.run(graph, "  test input  ")

print("\n✅ Execution Complete:")
print("   Input: '  test input  '")
print(f"   Final output: '{result.get('final_output')}'")

## Cleanup

In [ ]:
import shutil

shutil.rmtree(temp_dir)
print(f"🧹 Cleaned up temporary directory: {temp_dir}")

## Summary: YAML Include Best Practices

### ✅ When to Use Includes

**Good Use Cases:**
- Shared validation nodes across multiple pipelines
- Environment-specific configurations (dev/staging/prod)
- Organization-wide component libraries
- Common preprocessing steps
- Reusable output formatters

**Anti-Patterns to Avoid:**
- Over-fragmentation (too many small includes)
- Deep nesting (>3 levels)
- Circular dependencies
- Including everything (reduces readability)

### 🔒 Security Features

**Built-in Protections:**
1. **Circular Include Detection** - Prevents infinite loops
2. **Directory Traversal Protection** - Can't escape project root
3. **Absolute Path Rejection** - Only relative paths allowed
4. **Max Depth Limit** - Prevents excessive nesting (default: 10)

### 📁 Recommended Structure

```
project/
├── components/           # Reusable component library
│   ├── validation/
│   ├── processing/
│   └── output/
├── shared/              # Shared configurations
│   ├── common-nodes.yaml
│   └── base-config.yaml
├── environments/        # Environment configs
│   ├── dev.yaml
│   ├── staging.yaml
│   └── prod.yaml
└── pipelines/          # Main pipeline definitions
    ├── pipeline-a.yaml
    └── pipeline-b.yaml
```

### 🎯 Key Takeaways

1. **DRY Principle** - Include shared components to avoid duplication
2. **Composition** - Build complex pipelines from simple, reusable parts
3. **Environment Separation** - Use includes for dev/staging/prod configs
4. **Security** - Built-in protections prevent common vulnerabilities
5. **Maintainability** - Update once, affect all pipelines that include it

### 🔗 Related Topics

- Environment variables in YAML (`${VAR}` syntax)
- Jinja2 templates in YAML (`{{ variable }}` syntax)  
- Multi-document YAML for environment configs
- YAML validation and schema checking

The `!include` directive enables enterprise-scale pipeline composition patterns while maintaining security and simplicity!